In [1]:
from __future__ import print_function
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.svm import SVR
import statistics as st
from sklearn.linear_model import LinearRegression
from numpy import percentile
np.set_printoptions(threshold=sys.maxsize)

In [3]:
data = pd.read_csv('RHoMIS_Indicators_UTF-8.csv', header=None)
country_names = pd.read_csv('Country_Names.csv', header=None)
print(data.shape)
print(country_names.shape)
# print(data[10].values, data[11].values)

# name_location = list(zip(data[10].values, data[11].values))
# print(name_location[1:])

# coordinates_altitude = list(zip(data[7].values, data[8].values, data[12].values))
# print(coordinates_altitude[1:])


# print(country_names[1:])
print(len(data[10]))

countries = country_names[1][1:]
# print(countries)

(35729, 54)
(34, 3)
35729


In [4]:
# print(data[10][1])
# print(countries)
indexes = []
spec_index = []
lat_long = []
PPI_likelihood = []
Total_income = []
x_list = []
y_list = []

for i in range(1, len(countries)):
    for j in range(len(data[10])):
        if (countries[i] == data[10][j]):
            indexes.append((countries[i], j)) 

for i in range(len(indexes)):
    if (indexes[i][0] == 'Burkina_Faso'):
        spec_index.append(i)
        
for i in range(int(spec_index[0]) ,len(spec_index)):
    lat_long.append((data[7][i], data[8][i], data[9][i]))
    
# print(lat_long[1:])
# PPI = data[24].dropna().unique().tolist()

# numpy.cumsum(numpy.asarray(row[1], dtype=float)
# PPI = np.cumsum(np.asarray(data[24], dtype=float))
PPI = data[24]
Income = data[35]

# print(PPI)

for i in range(int(spec_index[0]) ,len(spec_index)):
    PPI_likelihood.append(PPI[i])
    Total_income.append(Income[i])

# print(PPI_likelihood[1:])
# print(Total_income[1:])

x = Total_income[1:]
# y = np.cumsum(np.asarray(PPI_likelihood[1:], dtype=float))
y = PPI_likelihood[1:]

# print(y)

for i in range(len(y)):
    if (str(y[i]) != 'nan') and (str(x[i]) != 'nan'):
        y_list.append(y[i])
        x_list.append(x[i])
        
# for i in range(len(y)):
#     if (str(y[i]) != 'nan'):
#         y_nan_list.append(i)

# print(y_nan_list[0])

# for i in range(len(y_nan_list)):
#     y.pop(y_nan_list[i])
    
# print(y)

print(len(y_list))
print(len(x_list))

3176
3176


In [5]:
# incoms=data['int_income'].dropna().unique().tolist()
# a_column = data['int_income'].dropna().unique().tolist()
# b_column = y[np.logical_not(pd.isnull(y))]

# a = [x for x in x if str(x) != 'nan']
# b = [y for y in y if str(y) != 'nan']

income = []
PPIs = [] 

#Total Income
a_column = np.array(x_list).astype(np.float)

#PPI
b_column = np.array(y_list).astype(np.float)

q25, q75 = percentile(b_column, 25), percentile(b_column, 75)
iqr = q75 - q25

cut_off = iqr*1.5
lower, upper = q25 - cut_off, q75 + cut_off
index = 0
for i in range(len(a_column)):
    if a_column[i] >= lower and a_column[i] <= upper and int(a_column[i]) != 0:
        income.append(a_column[i])
        PPIs.append(b_column[i])
        
test = np.array([246.1417083,18.31752248,0,476.2555845,32.97154047,48.65591909,28.62112888,22.8969031,0,80.13916086,2.747628372,0,0,105.8981768
,91.58761241,28.62112888,0,14.31056444,0,0,0,0,2207.261459,36.63504496,25.18659341,290.0274393,128.2226574,16.9437083,54.95256744
,0,0,36.63504496,22.8969031,4.57938062,29.76597403,0,0,41.21442558,19.08075258,16.9437083,91.58761241,16.71473926,68.69070931,433.6673447
,5.037318682,0,26.5604076,198.4398269,0,206.0721279,7.632301034,0,41.21442558,27.47628372,0,68.69070931,0,54.95256744,20.60721279,109.9051349
,0,503.7318682,0])
    

# # calculate the outlier cutoff
# cut_off = iqr * 1.5
# lower, upper = q25 - cut_off, q75 + cut_off
# indexxx=0
# income3=[]
# ppi = []
# for income in income2:
#     if income >= lower and income <= upper:
#         income3.append(income2[indexxx])
#         ppi.append(ppi_likelihood2[indexxx])
#         indexxx += 1
#     else:
#         indexxx+=1


def linear(x, y, test):
    error = []
    new_Y = []
    new_X = []
    
    Matrix = np.vander(x, 2, True)
    lsr = np.linalg.inv(Matrix.T.dot(Matrix)).dot(Matrix.T).dot(y)
    Y_array = lsr[0] + np.multiply(lsr[1], test)

    for d in range(len(Y_array)):
        error.append((Y_array[d] - y[d]) ** 2)
    
#     threshold_error = (sum(error)/len(error)*5)
    
#     for i in range(len(error)):
#         if (error[i] < threshold_error):
#             new_Y.append(Y_array[i])
#             new_X.append(x[i])
    
    return Y_array, np.sum(error), error


def poly(x, y, test):
    error3 = []
    Matrix_3 = np.vander(x, 4, True)
    lsr_3 = np.linalg.inv(Matrix_3.T.dot(Matrix_3)).dot(Matrix_3.T).dot(y)
    Y_poly3 = (lsr_3[0] * np.power(test, 0)) + (lsr_3[1] * np.power(test, 1)) + (lsr_3[2] * np.power(test, 2)) + (lsr_3[3] * np.power(test, 3))

#     for d in range(len(y)):
#         error3.append((Y_poly3[d] - y[d]) ** 2)

    return Y_poly3

def unknown(x, y, test):
    error = []
    Matrix = np.vander(np.sin(x), 2, True)
    lsr = np.linalg.inv(Matrix.T.dot(Matrix)).dot(Matrix.T).dot(y)
    Y_array = lsr[0] + np.multiply(lsr[1], np.sin(test))

    for d in range(len(Y_array)):
        error.append((Y_array[d] - y[d]) ** 2)

    return Y_array, np.sum(error)

Y_array1, sum_error, error_array = linear(income, PPIs, test)
Y_array = poly(income, PPIs, test)

sinu, error = unknown(income, PPIs, test)


# print(len(a_column), len(b_column))
# print(str('Data represented: ') + str(len(Y_array)) + ' ' + str(len(X_array)))

print(Y_array1)
# print(error_arr)
print(Y_array)
print(sinu)
# print(a_column[200])
# print(str("Total error: ") + str(sum_error))
# print(str("Thershold error: ") + str(threshold))
# plt.scatter(a_column, b_column, s=20, c="blue", marker="o", label="Data")
# plt.plot(X_array, Y_array, 'r-', lw=2, label="Best Fit")
# plt.xlabel('x-axis')
# plt.ylabel('y-axis')
# plt.grid(True)
# plt.title("Least Squares Best Fit Data Plot")
# plt.legend(loc='best', borderaxespad=0.)
# plt.show()

[42.67904066 46.51791212 46.8265651  38.80158758 46.27098973 46.00670562
 46.34429482 46.44074887 46.8265651  45.47620831 46.78026715 46.8265651
 46.8265651  45.04216505 45.28330019 46.34429482 46.8265651  46.58542996
 46.8265651  46.8265651  46.8265651  46.8265651   9.63388081 46.20925914
 46.40216725 41.93955956 44.66599423 46.54106109 45.90060616 46.8265651
 46.8265651  46.20925914 46.44074887 46.74940185 46.325004   46.8265651
 46.8265651  46.13209589 46.50505158 46.54106109 45.28330019 46.54491925
 45.66911642 39.51920576 46.74168553 46.8265651  46.37901828 43.48282447
 46.8265651  43.35421906 46.69795969 46.8265651  46.13209589 46.36358563
 46.8265651  45.66911642 46.8265651  45.90060616 46.4793305  44.97464721
 46.8265651  38.3386081  46.8265651 ]
[-1.00838391e+02  4.45595993e+01  4.39432186e+01 -1.41754187e+03
  4.55444157e+01  4.66757471e+01  4.52289828e+01  4.48389721e+01
  4.39432186e+01  4.75740214e+01  4.39694099e+01  4.39432186e+01
  4.39432186e+01  4.50557859e+01  4.6939

In [6]:
# # incoms=data['int_income'].dropna().unique().tolist()
# # a_column = data['int_income'].dropna().unique().tolist()
# # b_column = y[np.logical_not(pd.isnull(y))]

# # a = [x for x in x if str(x) != 'nan']
# # b = [y for y in y if str(y) != 'nan']

# #Total Income
# a_column = np.array(x_list).astype(np.float)

# #PPI
# b_column = np.array(y_list).astype(np.float)

# def linear(x, y):
#     error = []
#     new_Y = []
#     new_X = []
    
#     Matrix = np.vander(x, 2, True)
#     lsr = np.linalg.inv(Matrix.T.dot(Matrix)).dot(Matrix.T).dot(y)
#     Y_array = lsr[0] + np.multiply(lsr[1], x)

#     for d in range(len(Y_array)):
#         error.append((Y_array[d] - y[d]) ** 2)
    
#     threshold_error = (sum(error)/len(error)*0.05)
    
#     for i in range(len(error)):
#         if (error[i] < threshold_error):
#             new_Y.append(Y_array[i])
#             new_X.append(x[i])
    

#     return new_Y, new_X, np.sum(error), error, threshold_error

# Y_array, X_array, sum_error, error_arr, threshold = linear(a_column, b_column)

# print(len(a_column), len(b_column))
# print(str('Data represented: ') + str(len(Y_array)) + ' ' + str(len(X_array)))


# print(str("Total error: ") + str(sum_error))
# print(str("Thershold error: ") + str(threshold))
# plt.scatter(X_array, Y_array, s=20, c="blue", marker="o", label="Data")
# plt.plot(X_array, Y_array, 'r-', lw=2, label="Best Fit")
# plt.xlabel('x-axis')
# plt.ylabel('y-axis')
# plt.grid(True)
# plt.title("Least Squares Best Fit Data Plot")
# plt.legend(loc='best', borderaxespad=0.)
# plt.show()

In [8]:

# # #############################################################################
# # Generate sample data
# X = np.array(income).reshape(-1,1)
# y = np.array(PPIs).reshape(-1,1).ravel()

# # #############################################################################
# # Add noise to targets
# # y[::5] += 3 * (0.5 - np.random.rand(8))

# # #############################################################################
# # Fit regression model
# svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
# svr_lin = SVR(kernel='linear', C=100, gamma='auto')
# svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
#                coef0=1)

# # #############################################################################
# # Look at the results
# lw = 2

# svrs = [svr_rbf, svr_lin, svr_poly]
# kernel_label = ['RBF', 'Linear', 'Polynomial']
# model_color = ['m', 'c', 'g']

# fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 10), sharey=True)
# for ix, svr in enumerate(svrs):
#     axes[ix].plot(X, svr.fit(X, y).predict(X), color=model_color[ix], lw=lw,
#                   label='{} model'.format(kernel_label[ix]))
#     axes[ix].scatter(X[svr.support_], y[svr.support_], facecolor="none",
#                      edgecolor=model_color[ix], s=50,
#                      label='{} support vectors'.format(kernel_label[ix]))
#     axes[ix].scatter(X[np.setdiff1d(np.arange(len(X)), svr.support_)],
#                      y[np.setdiff1d(np.arange(len(X)), svr.support_)],
#                      facecolor="none", edgecolor="k", s=50,
#                      label='other training data')
#     axes[ix].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
#                     ncol=1, fancybox=True, shadow=True)

# fig.text(0.5, 0.04, 'data', ha='center', va='center')
# fig.text(0.06, 0.5, 'target', ha='center', va='center', rotation='vertical')
# fig.suptitle("Support Vector Regression", fontsize=14)
# plt.show()

# # print(X.shape)
# # print(np.array(income).reshape(-1,1).shape)